In [21]:
library(tidymodels)
library(tidyverse)
library(stacks)

subm<-read_csv("/kaggle/input/yassir-data/sample_submission.csv")
test<-read_csv("/kaggle/input/yassir-data/test.csv")
train<-read_csv("/kaggle/input/yassir-data/train.csv")

Rows: 276469 Columns: 2
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (1): id
dbl (1): trip_fee

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 276469 Columns: 18
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (8): id, pickup_city, destination_city, rider, driver, trip_status, rid...
dbl  (6): trip_distance, driver2rider_distance, used_discount, rider_rating,...
dttm (4): request_date, accepted_date, started_date, trip_finished_date

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 723406 Columns: 19
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (8): id, pickup_city, destination_city, rider, driver, trip_s

In [ ]:
library(lubridate)
recipe<-train %>% 
  recipe(trip_fee ~ .) %>% 
  step_mutate(time_acc_req = accepted_date - request_date, 
              time_to_arrive = started_date - accepted_date, 
              wday = wday(request_date),
             month=month(request_date),
             hour=hour(request_date)) %>% 
  step_mutate(time_acc_req = as.double(time_acc_req), 
              time_to_arrive = as.double(time_to_arrive)) %>%
 step_rm(id, rider, driver,accepted_date,started_date,trip_finished_date,request_date)%>%
  step_impute_mean(rider_rating, driver_rating) %>%
  step_impute_median(time_acc_req,time_to_arrive,wday)%>%
  step_impute_mode(rider_cancel_reason,driver_cancel_reason)%>%
  step_mutate(discount_amount = case_when(is.na(discount_amount) ~ 0, TRUE ~ discount_amount)) %>%
  step_normalize(all_numeric_predictors()) %>% 
  step_dummy(all_nominal_predictors()) %>%
  step_poly(all_numeric_predictors(),degree=2)%>%
  prep()



In [3]:
train_1<-recipe%>%bake(train)
test_1<-recipe%>%bake(test)


Warning message:
“There are new levels in a factor: Boumerd��s”


In [4]:
sample<-train_1%>% .[1:400000,]

# Model

In [5]:

library(finetune)
xgb_spec <- boost_tree() %>%
  set_engine("xgboost") %>%
  set_mode("regression")

set.seed(123)
vb_folds <- vfold_cv(train_1,v = 4)
## workflow

xgb_wf <- workflow() %>%
  add_model(xgb_spec) %>% 
  add_formula(trip_fee ~ .)






In [6]:
pred<-xgb_Wf%>% predict(test_1)

subm$trip_fee<-pred$.pred
write.csv(subm, "last3.csv", row.names = FALSE)

In [8]:

# Create a Poisson regression model specification
poisson_spec <- 
  linear_reg() %>% 
  set_engine("glm", family = "poisson")

# Create a recipe (optional but often useful for preprocessing)
poisson_recipe <- 
  recipe(trip_fee ~ ., data = train_1)

# Fit the model on the training data
poisson_fit <- 
  workflow() %>% 
  add_model(poisson_spec) %>% 
  add_formula(trip_fee ~ .) %>% 
  fit(train_1)

# Predict on the testing data
predictions <- 
  predict(poisson_fit, test_1)

In [20]:
subm$trip_fee<-predictions$.pred
write.csv(subm, "last6.csv", row.names = FALSE)